In [1]:
#all the imports used in the program

import pandas as pd 
import pyodbc
import numpy as np
import matplotlib.pyplot as plt
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.metrics import categorical_accuracy
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM
from keras.layers import GlobalMaxPool1D, Conv1D, Dropout, GRU, Flatten, MaxPooling1D
from keras.optimizers import Adam
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import sklearn.metrics as skm
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
#Grab data from a database

def getData(Server, Database, query):    
    
    #create a SQL connection based on the given server and database
    sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                          'SERVER='+Server+';' 
                          'DATABASE='+Database+';' 
                          'Trusted_Connection=yes')
    
    #return the data from the given Query and SQL connection,
    #here i hard coded the index so all queries must select examCode
    #for other instances just simply change or remove depending on use
    return pd.read_sql(query, sql_conn)

In [3]:
#establish my server and corresponding database to pull data from
server ='GESTALT-BT41Q'
database = 'MClinical'

#this query grabs sectionValues with their corresponding examCode and description
#does not select examCodes if there is less than 100 section values for the corresponding examCode
#Stores the result in a pandas DataFrame object called data
query = "SELECT CPTCODEKEY.CPT88304, CPTCODEKEY.CPT88305, CPTCODEKEY.CPT88307, CPTCODEKEY.CPT88309, CPTCODEKEY.CPT88331, CPTCODEKEY.CPT88341, CPTCODEKEY.CPT88342, CPTCODEKEY.CPT88112, CPTCODEKEY.CPT88141, CPTCODEKEY.CPT88175, description, ResultSection.sectionValue FROM [MClinical].[dbo].[Result] LEFT JOIN ResultSection ON Result.resultKey = ResultSection.resultKey left join mapResultRequestedProcedure ON Result.resultKey = mapResultRequestedProcedure.resultKey  left join RequestedProcedure ON mapResultRequestedProcedure.requestedProcedureKey = RequestedProcedure.requestedProcedureKey left join FillerOrder ON RequestedProcedure.fillerOrderKey = FillerOrder.fillerOrderKey left join PlacerOrder ON FillerOrder.placerOrderKey = PlacerOrder.placerOrderKey left join ExamCode ON PlacerOrder.examCodeKey = ExamCode.examCodeKey left join Patient on FillerOrder.patientKey = Patient.patientKey left join cptcodekey on patient.patientkey = cptcodekey.patient_key WHERE sectionValue <> ' ' and (ResultSection.sectionValue <>' No diagnosis; performed technical only ') and ResultSection.sectionCategory like '%gross%' and examCode not like '%:%' and description is not null and Patient.patientKey in ( select patient_key from cptcodekey ) order by patient.patientkey"



original = getData(server,database,query)
data = original.copy()


In [4]:
#removes words that have at one colon somewhere in the middle of two words
#and words that have two colons separated by three words. 

def removeColon(df):
    
    #Word array of words that i will later remove
    bagOfWords = []
    
    #a array of every word in the sectionValue on the given dataframe df
    wordList = df.sectionValue.str.split(expand=True).stack()
    
    for word in wordList:
        colonWord = re.search(r"\w+:\w+:\w+", word)
        if colonWord is None:
            colonWord = re.search(r"\w+:\w+", word)
        if colonWord is not None:
            if colonWord.group() not in bagOfWords:
                bagOfWords.append(colonWord.group())
    
    #return the updated dataframe sectionValue, only keeping words that are not contained in bagOfWords            
    return df['sectionValue'].apply(lambda x: ' '.join([word for word in x.split() if word not in (bagOfWords)]))

This method is more useful than it looks.

You pass in a pandas DataFrame and tweak it.

First, i create a empty list called bagOfWords

The next line seemes confusing but bassically what I am doing here is taking every word from the sectionValue column and creating a list in which each row only contains one word. This helps with the iterative process. I store the result of this into WordList(approx 1.4 million words)

Next, i iterate through every word in the wordList in a for loop.

let me explain how the search process works and what words i am looking to add to my bagOfWords
1. How does the search processWork?
    1. I use Regex(regular expression) to do my searching for me which is on a character by character basis
2. What kind of words am i looking for?
    1. \w+:\w+ and \w+:\w+:\w+
        1. \w searches for any character in the form [a-zA-Z0-9]
        2. \+ searches for the previous search condition until the end of the word. 
        3. : specifies that i want a colon 
        4. putting it all together 
            1. \w+:\w+ searches for a character in the form [a-zA-Z0-9] for any amount of characters in that form until it hits a colon : in which then it does the same \w+ until the end of the word.
            2. \w+:\w+:\w+ is the same as above just has two colons i hope you get the picture..
            
     
Since i am searching for two different types of words i need to search two different times for every word in wordList.

I search the word to see if it matches the pattern of having three words separated by 2 colons, this returns a match object which i store in colonWord.

if the word isnt found in the search it returns None, so i check if colonWord is None. If it is i search for the different type of word and store that searches result into colonWord.

After that process is done i finally check to see if either of my searches came back true(not None)

If they do i use colonWord.group() function to grab just the string(word) that it found.

Then Check the bagOfWords to see if the word i found is already in it. 

If the word is already in it I move onto the next word in the wordList.

if it is not, i simply  add it and move to the next word as well. 

Finally once i have scanned all words and created my bagOfWords that is a unique list i remove those words from the sectionValue column of the Data. 

what the last line in the funtion is doing is recreating my column sectionValue, but only keeping words that are NOT in bagOfWords.

Once that is done i return the new column of sectionValue. 

In [5]:
#here i wanted to remove punctuation from the column sectionValue in my pandas dataFrame
#i replace every character that matches with one of the following below with nothing.

def removePunctuation(df1):
    df1['sectionValue'] = df1['sectionValue'].str.replace(',', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('.', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('?', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('/', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('/', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('+', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('-', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('=', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('_', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace(')', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('(', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('*', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('&', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('^', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('%', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('$', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('#', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('@', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('!', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('>', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('<', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('[', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace(']', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('{', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('}', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('|', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace(':', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace(';', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('\'', '')
    df1['sectionValue'] = df1['sectionValue'].str.replace('\"', '')
    return df1
    

In [6]:
def removeStopWords(df):
    
    #stopWords are words that have relatively no meaning to any actual data
    #we dont want that here so lets remove them
    stop = stopwords.words('english')
    
    # add custom stopWords 
    stop = addStopWords(stop)
    
    # add this if you want to remove words that are smaller than size two
    # change the size to whatever you like 
    
    #df['sectionValue'] = df['sectionValue'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))
    
    #returning the new sectionValue to the Datafram with words that are not in the StopWords
    return df['sectionValue'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [7]:
#adding my own custom stopWords
#super simple to add as you can see 
#modify as needed!

def addStopWords(stop):
    stop.append('-')
    stop.append('a')
    stop.append('b')
    stop.append('c')
    stop.append('d')
    stop.append('e')
    stop.append('f')
    stop.append('g')
    stop.append('h')
    stop.append('i')
    stop.append('j')
    stop.append('k')
    stop.append('l')
    stop.append('m')
    stop.append('n')
    stop.append('o')
    stop.append('p')
    stop.append('q')
    stop.append('r')
    stop.append('s')
    stop.append('t')
    stop.append('u')
    stop.append('v')
    stop.append('w')
    stop.append('x')
    stop.append('y')
    stop.append('z')
    stop.append('no')
    stop.append('see')
    stop.append('two')
    stop.append('0')
    stop.append('1')
    stop.append('2')
    stop.append('3')
    stop.append('4')
    stop.append('5')
    stop.append('6')
    stop.append('7')
    stop.append('8')
    stop.append('9')
    return stop

In [8]:
#clean_text transforms words like tomatoes, tomato, tomatos, all to tomato. this is very helpful.

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = " ".join(text)
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\csorensen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
#concatinate the sectionValue and description feild
def concatExamDesc(df):
    return  df['description'] + ' ' + df['sectionValue']

In [10]:
def clean(df):
    #change the case of all the words to lower case so there is no case sensitivity.
    df['sectionValue'] = df['sectionValue'].str.lower()

    #call the removeColon function to remove words with a colon or mulitple colons in the middle of the word
    df['sectionValue'] = removeColon(df)

    #call the removePuncuation code, notice how i did this after the removeColon function.
    #it is important that we call this after the removeColon Function because this would remove colons
    #from words we want to remove, and then the remove colon function would never find anything because there is no colons. 
    df = removePunctuation(df)

    #call the removeStopWords function to remove words that have no meaning.
    df['sectionValue'] = removeStopWords(df)

    #call the clean_text to place words of simularity with the base word (ex: biopsies -> biopsy)
    df['sectionValue'] = df.sectionValue.apply(lambda x: clean_text(x))

    #add the description to the sectionValue
    df['sectionValue'] = concatExamDesc(df)
    return df['sectionValue']

In [11]:
#call the clean data function to clean data
data['sectionValue'] = clean(data)

In [12]:
def convert_text(df, maxlen, max_words):
    #split df into two series
    #texts being the sectionValue
    #labels being the cooresponding examCode
    texts = df.sectionValue
    
    #convert the series into numpy arrays
    texts = texts.values
    
    #create a tokenizer based on the max_words
    #fit the tokenizer to our specific texts
    #change our texts to a vetorized integer
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))
    
    #pad sequences ensures that all our vectors are of the same length
    x = pad_sequences(sequences, maxlen=maxlen)
    
    
    #create a dictionary to map all
    #cpt codes to 1 and keep other values as 0
    d = defaultdict(LabelEncoder)
    
    fit = df[['CPT88304', 'CPT88305', 'CPT88307',
            'CPT88309', 'CPT88331', 'CPT88341', 
            'CPT88342', 'CPT88112', 'CPT88141', 
            'CPT88175']].apply(lambda y: d[y.name].fit_transform(y))   
    labels = fit.values

    print('Shape of data tensor:', x.shape)
    print('Shape of label tensor:', labels.shape)
    
    #return x, labels, and the last 7000 of x and labels for testing
    return x[:13000], labels[:13000], x[-1419:], labels[-1419:]

In [13]:
#define maxlen as the maximum words to take from each sectionValue
#define max_words as the total number of unique words to tokenize

maxlen = 150
max_words = 20000

#create data that can be ran through our model
x_train, y_train, x_test, y_test = convert_text(data, maxlen, max_words)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

Found 16999 unique tokens.
Shape of data tensor: (14419, 150)
Shape of label tensor: (14419, 10)
(13000, 150)
(13000, 10)
(1419, 150)
(1419, 10)


In [14]:
#create a machine learning model with the following
def create_model(max_words,maxlen):
    #keras default model
    model = Sequential()
    
    #add an embedding layer with the input dim and input length to what we have already
    #configured for our vectorized forms of our text
    model.add(Embedding(input_dim = max_words, input_length=maxlen, output_dim = 50))
    
    #model.add(Bidirectional(LSTM(64)))
    
    model.add(Dropout(0.15))
    model.add(Conv1D(maxlen, 3, padding='valid', activation='relu', strides=1))
    #model.add(Dropout(0.5))
    #model.add(Conv1D(maxlen, 3, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPool1D())
    # create a dense output layer with the units = len(labels_dict)
    model.add(Dense(10, activation='sigmoid'))
    
    #print the summary
    model.summary()
    
    return model

In [15]:
#create the model
model = create_model(max_words, maxlen)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 50)           1000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 150)          22650     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 150)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1510      
Total params: 1,024,160
Trainable params: 1,024,160
Non-trainable params: 0
___________

In [16]:
#train the model
def train_model(model, x_train, y_train, epochs, batchsize, max_words, max_len):
    #compile the model
    #optimizer -> adam (better for multi-label applications)
    #loss -> sbinary_crossentropy(we are using this because each label is a unique binary case)
    #meteric -> accuracy
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc']) 
    #save the history from the model
    #set the paramiters
    #fit the model 
    history = model.fit(x_train, 
                        y_train,
                        epochs=epochs,
                        batch_size=batchsize)
    return history

In [17]:
#train the model
history = train_model(model, x_train, y_train, 21, 500, max_words, maxlen)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/21
13000/13000 [==============================] - 13s 1ms/step - loss: 0.5650 - acc: 0.8421
Epoch 2/21
13000/13000 [==============================] - 12s 898us/step - loss: 0.3171 - acc: 0.8938
Epoch 3/21
13000/13000 [==============================] - 12s 908us/step - loss: 0.2715 - acc: 0.9044
Epoch 4/21
13000/13000 [==============================] - 12s 929us/step - loss: 0.2531 - acc: 0.9093
Epoch 5/21
13000/13000 [==============================] - 12s 901us/step - loss: 0.2320 - acc: 0.9171
Epoch 6/21
13000/13000 [==============================] - 12s 909us/step - loss: 0.2086 - acc: 0.9263
Epoch 7/21
13000/13000 [==============================] - 13s 992us/step - loss: 0.1866 - acc: 0.9342
Epoch 8/21
13000/13000 [==============================] - 12s 899us/step - loss: 0.1713 - acc: 0.9403
Epoch 9/21
13000/13000 [==============================] - 12

In [18]:
#test the model on our set aside testing data
def test_model(model, x_test):
    #gather the models prediction 
    
    #the model displays its prediction as a list of all the cpt codes 
    #with percents in each category at how confident the model is for 
    #each cptCode. 
    
    #since we used a binary classifier
    #anything above .5 will  be considered true
    #and anything below .5 will be considered false
    preds = model.predict(x_test)
    
    #for every row in the prediction list
    #change every column value for the specific row
    #where the percent is above or equal to .5 to 1 
    #and below .5 to 0
    preds[preds>=0.5] = 1
    preds[preds<0.5] = 0
    
    #convert the list to a numpy array
    return np.asarray(preds)

In [19]:
#test the model against our test data and store the predictions in y_pred
y_pred = test_model(model, x_test)

print('Test accuracy : ',(1 - skm.hamming_loss(y_test, y_pred)) * 100)

Test accuracy :  96.19450317124736


In [20]:
#drop first n rows
def drop_first_n_rows(df, n):
    return df.iloc[n:]

In [21]:
#drop frist 13000 rows in data
data = drop_first_n_rows(data, 13000)

In [22]:
#convert 1's in columns to cooresponding cpt code
def convert_to_cpt_code(Y):
    #create a label dict
    temp = ['88304', '88305', '88307',
            '88309', '88331', '88341', 
            '88342', '88112', '88141', 
            '88175']
    label_dict = np.asarray(temp)
    
    #create a container for the 1419 x 10 matrix (big storage container)
    temp = []
    
    #access every row in Y
    for row in Y:
        #create a container for each row (mini storage container lol)
        temp1 = []
        i = 0
        
        #grab every column value in the cooresponding row
        for index in row:
            #if the value is 1 then append into our mini storage container
            if (index == 1):
                temp1.append(label_dict[i])
            #append a 0    
            else:
                temp1.append('0')
            
            #increase the label dict tracker
            i = i + 1
            
        #append the mini storage container to the big storage container
        #NUMPY ARRAY !!!
        temp.append(np.asarray(temp1))
    
    #numpy array!!!!!!
    return np.asarray(temp)         

In [23]:
pred = convert_to_cpt_code(y_pred)
test = convert_to_cpt_code(y_test)

In [24]:
temp = ['88304', '88305', '88307',
        '88309', '88331', '88341', 
        '88342', '88112', '88141', 
        '88175']
label_dict = np.asarray(temp)

pdpred = pd.DataFrame(pred, columns=label_dict)
pdtest = pd.DataFrame(test, columns=label_dict)

#print(pdpred, '\n\n', pdtest)

In [25]:
def zero_to_nan(df):
    df['88304'].replace('0', np.nan, inplace=True)
    df['88305'].replace('0', np.nan, inplace=True)
    df['88307'].replace('0', np.nan, inplace=True)
    df['88309'].replace('0', np.nan, inplace=True)
    df['88331'].replace('0', np.nan, inplace=True)
    df['88341'].replace('0', np.nan, inplace=True)
    df['88342'].replace('0', np.nan, inplace=True)
    df['88112'].replace('0', np.nan, inplace=True)
    df['88141'].replace('0', np.nan, inplace=True)
    df['88175'].replace('0', np.nan, inplace=True)
    return df

In [26]:
pdpred = zero_to_nan(pdpred)
pdtest = zero_to_nan(pdtest)

In [27]:
pdtest['truth'] = pdtest[['88304', '88305', '88307', '88309', '88331', '88341', '88342', '88112', '88141', '88175']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)
pdpred['pred'] = pdpred[['88304', '88305', '88307', '88309', '88331', '88341', '88342', '88112', '88141', '88175']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)


In [28]:
data['pred'] = pdpred['pred'].values
data['truth'] = pdtest['truth'].values

In [29]:
data = data.drop(['description', 'CPT88304', 'CPT88305', 'CPT88307',
                  'CPT88309', 'CPT88331', 'CPT88341', 
                  'CPT88342', 'CPT88112', 'CPT88141', 
                  'CPT88175'], axis =1)

In [30]:
data = data.reindex(columns=['truth', 'pred', 'sectionValue'])

In [31]:
data

,truth,pred,sectionValue
13000,88305,88305,"A. BREAST NEEDLE BIOPSY, RIGHT specimen receiv..."
13001,"88304, 88305",88304,A. LIPOMA label designate clifford back receiv...
13002,88305,88305,"A. BREAST NEEDLE BIOPSY, RIGHT specimen label ..."
13003,"88305, 88331","88305, 88331",A. SYNOVIUM specimen receive three part specim...
13004,88305,"88305, 88342","A. SKIN, OTHER THAN CYST/TAG/DEBRIDEMENT/PLAST..."
13005,88305,"88305, 88342",A. SINUS CONTENTS specimen receive part specim...
13006,"88307, 88341, 88342","88307, 88342",A. LIVER NEEDLE BIOPSY specimen receive contai...
13007,88304,88304,A. GALLBLADDER formalin label designate kitche...
13008,"88305, 88307, 88331, 88341, 88342",88305,"A. BREAST BIOPSY, WITHOUT SURGICAL MARGINS, LE..."
13009,"88305, 88307, 88331, 88341, 88342","88305, 88307, 88309, 88331, 88341, 88342","A. LYMPH NODE, SENTINEL five specimen receive ..."


In [32]:
def getsqlconn(Server, Database):    
    
    #create a SQL connection based on the given server and database
    sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                          'SERVER='+Server+';' 
                          'DATABASE='+Database+';' 
                          'Trusted_Connection=yes')
    
    #return the data from the given Query and SQL connection,
    #here i hard coded the index so all queries must select examCode
    #for other instances just simply change or remove depending on use
    return sql_conn

In [33]:
server ='GESTALT-BT41Q'
database = 'CPTCode'

connStr = getsqlconn(server,database)

In [34]:
cursor = connStr.cursor()

for index,row in data.iterrows():
    cursor.execute("INSERT INTO dbo.MachineLearning([idx],[Pred],[Truth],[Val]) values (?,?, ?,?)" ,index, row['pred'], row['truth'] , row['sectionValue']) 
    connStr.commit()
cursor.close()
connStr.close()